<a href="https://colab.research.google.com/github/dxiong2001/malibu-ml/blob/html_parser/html_parser_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install beautifulsoup4

In [2]:
from bs4 import BeautifulSoup
import requests
import nltk
from nltk.tokenize import sent_tokenize
import re
import unicodedata
import spacy

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
#url='https://www.npr.org/2022/07/14/1111618620/secret-service-erased-texts-from-two-day-period-spanning-jan-6-attack-watchdog-s'
#url = "https://www.popsci.com/environment/climate-change-wildlife-migration/"
url = "https://www.popsci.com/science/omicron-coronavirus-variant/"
page = requests.get(url)
#page.content

In [47]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())
#list(soup.children)
#grab body text
body_text = soup.find('div', class_='Article-bodyText')
body_text_p = body_text.find_all('p', class_='')

#grab article information
header = soup.find('div', class_='Article-header')
article_info = header.find('p', class_='Article-author').get_text()
title = header.find('h1', class_='u-entryTitle').get_text()
author = article_info.split("|")[0].replace("\n                    ","").replace("\n              ","")[2:]
date = article_info.split("|")[1].replace("\n    \n    Updated ","").replace("\n  \n","")
print(title)
print(author)
print(date)


processed_text1 = [item.get_text() for item in body_text_p]
print(processed_text1)

processed_text2=[]
for processed in processed_text1:
  p2 = processed.replace('“', '"').replace('”', '"').replace("‘", "'").replace("’", "'").replace('…', '...').replace('–', '-')
  #processed_text2.append(unicodedata.normalize('NFKD', p2).encode('ascii', 'ignore').decode('utf-8'))
  processed_text2.append(unicodedata.normalize('NFKD', p2))

print(processed_text2)

Omicron may not be more dangerous than other variants, early hospital report suggests
Hannah Seo
Dec 6, 2021 1:06 PM
['Preliminary clinical data from South Africa suggest that the COVID-19 Omicron variant may be less dangerous than initially feared.\xa0', 'The South African Medical Research Council posted a report on Saturday outlining early Omicron cases from several hospitals. They found that patients infected with the Omicron strain tended to have less severe disease—most of the hospitalized patients did not need supplemental oxygen, a few developed COVID pneumonia, and fewer still required intensive care.\xa0', '“The signals are a bit encouraging” but Omicron’s risk profile is still incomplete, Anthony Fauci, head of the National Institute of Allergy and Infectious Diseases, told CNN on Sunday. More clinical data from around the world will need to be assessed for epidemiologists to get a true sense of where Omicron stands in relation to other variants. “Clearly, in South Africa, Om

In [12]:
quotes = []

#for p in processed_text:
#  quotes.append(re.findall('"([^"]*)"', p))

sentences = [sent_tokenize(sent) for sent in processed_text2]

combined_sentences = []
for sent in sentences:
  combined_sentences += sent
# i=0
# for sent in sentences:
#   combined_sentence = ""
#   for s in sent:
#     combined_sentence += s
#   sentences[i] = combined_sentence
#   i=i+1


quotes_incomplete = [re.findall('"([^"]*)"', paragraph) for paragraph in processed_text2]
quotes_incomplete = [q for q in quotes_incomplete if q != []]
quotes_incomplete2 = [j for i in quotes_incomplete for j in i]

quotes_complete =[]

i=0
for comb in combined_sentences:
  
  if(quotes_incomplete2[i] in comb):
    if(comb not in quotes_complete):
      quotes_complete.append(comb)
    i = i + 1
    while i < len(quotes_incomplete2) and quotes_incomplete2[i] in comb:
        i = i + 1
    
  
  if(i==len(quotes_incomplete2)):
      break

print(quotes_complete)
#print(quotes)

['Preliminary clinical data from South Africa suggest that the COVID-19 Omicron variant may be less dangerous than initially feared.', 'The South African Medical Research Council posted a report on Saturday outlining early Omicron cases from several hospitals.', 'They found that patients infected with the Omicron strain tended to have less severe diseasemost of the hospitalized patients did not need supplemental oxygen, a few developed COVID pneumonia, and fewer still required intensive care.', '"The signals are a bit encouraging" but Omicron\'s risk profile is still incomplete, Anthony Fauci, head of the National Institute of Allergy and Infectious Diseases, told CNN on Sunday.', 'More clinical data from around the world will need to be assessed for epidemiologists to get a true sense of where Omicron stands in relation to other variants.', '"Clearly, in South Africa, Omicron has a transmission advantage," Fauci added, "although it\'s too early to make any definitive statements about 

In [45]:
##need to handle additional p tag text not relevant to article

joined_sentences = ' '.join(combined_sentences)
print(joined_sentences)
nlp_processed_text = nlp(joined_sentences)

named_entities=[]
for entities in nlp_processed_text.ents:
  named_entities.append((entities.text, entities.label_))
  print(entities.text, entities.label_)

people = []
for n in named_entities:
  if(n[1]=="PERSON" and " " in n[0] and not any(i.isdigit() for i in n[0])):
    people.append(n[0])

people_extended = []
for p in range(len(people)):
  temp = []
  temp.append(people[p])
  temp = temp+people[p].split()
  people_extended.append(temp)
print(people)
print(people_extended)

Preliminary clinical data from South Africa suggest that the COVID-19 Omicron variant may be less dangerous than initially feared. The South African Medical Research Council posted a report on Saturday outlining early Omicron cases from several hospitals. They found that patients infected with the Omicron strain tended to have less severe diseasemost of the hospitalized patients did not need supplemental oxygen, a few developed COVID pneumonia, and fewer still required intensive care. "The signals are a bit encouraging" but Omicron's risk profile is still incomplete, Anthony Fauci, head of the National Institute of Allergy and Infectious Diseases, told CNN on Sunday. More clinical data from around the world will need to be assessed for epidemiologists to get a true sense of where Omicron stands in relation to other variants. "Clearly, in South Africa, Omicron has a transmission advantage," Fauci added, "although it's too early to make any definitive statements about it, thus far it doe

In [7]:
tokens = nltk.word_tokenize(quotes_complete[1])
pos_tag = nltk.pos_tag(tokens)
print([p[1]=='NNP' for p in pos_tag])

[False, False, False, False, True, True, False, True, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


In [9]:
# def attribute_quote(people, quotes):
#   for q in quotes:

